# Simulation2

**Mengqi Liu**

**Jun 16, 2023**
___

In [1]:
import numpy as np
import pandas as pd
import math
from itertools import permutations, combinations
import statsmodels.formula.api as smf
import multiprocessing as mp
from tqdm import tqdm
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as ssm

In [50]:
def data_generative1(N=100, s=1):
    np.random.seed(s); Z = np.random.uniform(0, 10, N)

    def fz(Z):
        return np.array([np.exp(math.sin(z) - np.log(4)) for z in Z])
    
    np.random.seed(s); X = np.random.binomial(1, fz(Z))
    np.random.seed(s+100); Y = np.random.binomial(1, fz(Z))
    return X, Y, Z
X, Y, Z = data_generative1(N=100)

In [51]:
modx = smf.ols(formula='X ~ Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()

In [12]:
modx = smf.ols(formula='X ~ Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()
mody = smf.ols(formula='Y ~ Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()
T = np.abs(np.corrcoef(modx.resid, mody.resid)[0, 1])

In [25]:
def discretize(X, M = 10):
    '''Discretize continuous variable X

    Args: 
        M: num of bins
    
    Returns:
        new_X: (np.array) discretized X with values of midpoints in all bins
    '''
    bins = np.linspace(np.min(X), np.max(X), M+1)
    mid_bins = (bins[0:-1] + bins[1:]) / 2
    bins[0] -= 1 # pd.cut doesn't include left boundary value except add right=False
    G = np.array(pd.cut(X, bins, labels=[x for x in range(M)]))
    print(bins)
    print(mid_bins)
    new_X = np.array([mid_bins[x] for x in G])
    return new_X

In [41]:
import sys
import os

In [64]:
def compute_G(Z, M = 10):
    '''Compute bin edges for continuous Z

    Args: 
        M: num of bins
    
    Returns:
        G: (np.array) [len(Z)] with element of {0,1,...,M-1}
    '''
    bins = np.linspace(np.min(Z), np.max(Z), M+1)
    bins[0] -= 1 # pd.cut doesn't include left boundary value except add right=False
    G = np.array(pd.cut(Z, bins, labels=[x for x in range(M)]))
    return G

def discretize(X, M = 10):
    '''Discretize continuous variable X

    Args: 
        M: num of bins
    
    Returns:
        new_X: (np.array) discretized X with values of midpoints in all bins
    '''
    bins = np.linspace(np.min(X), np.max(X), M+1)
    mid_bins = (bins[0:-1] + bins[1:]) / 2
    bins[0] -= 1 # pd.cut doesn't include left boundary value except add right=False
    G = np.array(pd.cut(X, bins, labels=[x for x in range(M)]))
    new_X = np.array([mid_bins[x] for x in G])
    return new_X

def compute_T_linear(X, Y, Z):
    '''Compute testing statistic from linear regression

    Args:
        X, Y, Z: data
    
    Returns:
        T: computed testing statistic
    '''
    mod = smf.ols(formula='Y ~ X + Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()
    T = np.abs(mod.params['X'])
    return T

def compute_T_double(X, Y, Z):
    '''Compute testing statistic from double linear regression

    Args:
        X, Y, Z: data
    
    Returns:
        T: computed testing statistic
    '''
    modx = smf.ols(formula='X ~ Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()
    mody = smf.ols(formula='Y ~ Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()
    T = np.abs(np.corrcoef(modx.resid, mody.resid)[0, 1])
    return T

def LPT(X, Y, Z, G, B=100, M=10, alpha=0.05, cont_z=True, \
        cont_xy=False):
    '''Local permutation test

    Args:
        X(discrete), Y(discrete), Z(discrete or continuous): data
        G: categories of bins
        B: num of permutation tests
        M: num of bins
        l1: number of possible values for X
        l2: number of possible values for Y
        alpha: confidence level
        cont: whether Z is continuous
    
    Returns:
        p1: p-value for testing statistic from compute_T
        p2: p-value for testing statistic from compute_T_linear
    '''
    T_sam = compute_T(X, Y, Z, G, M, cont_z, cont_xy)
    T_per = np.zeros(B)

    T_sam_linear = compute_T_linear(X, Y, Z)
    T_per_linear = np.zeros(B)

    T_sam_double = compute_T_double(X, Y, Z)
    T_per_double = np.zeros(B)

    def perm_Y(Y, G, s):
        '''Permutate Y within each bin'''
        new_Y = Y.copy()
        for m in range(M):
            m_ind = list(np.where(G == m)[0])
            m_ind_new = m_ind.copy()
            np.random.seed(s); np.random.shuffle(m_ind_new)
            new_Y[m_ind] = new_Y[m_ind_new]
        return new_Y

    for b in range(B):
        new_Y = perm_Y(Y, G, b)
        T_per_linear[b] = compute_T_linear(X, new_Y, Z)
        T_per_double[b] = compute_T_double(X, new_Y, Z)

    N = len(G)
    NN = np.random.poisson(lam=N/2, size=1)
    if NN > N:
        p = 1.0
    else:
        NN_ind  = np.random.choice(N, NN, replace=False)
        for b in range(B):
            new_Y = perm_Y(Y[NN_ind], G[NN_ind], b)
            T_per[b] = compute_T(X[NN_ind], new_Y, Z[NN_ind], G[NN_ind], M, cont_z, cont_xy)
        p = (T_per >= T_sam).sum() / B
    p_linear = (T_per_linear >= T_sam_linear).sum() / B
    p_double = (T_per_double >= T_sam_double).sum() / B
    #return int(p <= alpha), int(p_linear <= alpha)
    return p, p_linear, p_double

In [65]:
LPT(X, Y, Z, G, B=100, M=10, alpha=0.05, cont_z=True, \
        cont_xy=True)

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


(0.06, 0.83, 0.83)